In [1]:
from joblib import load
import pandas as pd
import numpy as np
import os
from  data_utilities import *
import cv2  # OpenCV 用于调整图像大小和颜色处理
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc  # 引入垃圾回收模块
from tqdm.auto import tqdm  # 自动适配环境 导入tqdm进度条库
from collections import defaultdict

In [2]:
dataset_name = 'ManyTx'
dataset_path='../../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])


tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
equalized = 1
capture_date_list = compact_dataset['capture_date_list']


n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


数据集发射机数量： 150 具体为： ['1-1', '1-10', '1-11', '1-12', '1-14', '1-15', '1-16', '1-18', '1-19', '1-2', '1-8', '10-1', '10-10', '10-11', '10-17', '10-4', '10-7', '11-1', '11-10', '11-17', '11-19', '11-20', '11-4', '11-7', '12-1', '12-19', '12-20', '12-7', '13-14', '13-18', '13-19', '13-20', '13-3', '13-7', '14-10', '14-11', '14-12', '14-13', '14-14', '14-20', '14-7', '14-8', '14-9', '15-1', '15-19', '15-6', '16-1', '16-16', '16-19', '16-20', '16-5', '17-10', '17-11', '18-1', '18-10', '18-11', '18-12', '18-13', '18-14', '18-15', '18-16', '18-17', '18-2', '18-20', '18-4', '18-5', '18-7', '18-8', '18-9', '19-1', '19-10', '19-11', '19-12', '19-13', '19-14', '19-19', '19-2', '19-20', '19-3', '19-4', '19-6', '19-7', '19-8', '19-9', '2-1', '2-12', '2-13', '2-14', '2-15', '2-16', '2-17', '2-19', '2-20', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '20-1', '20-12', '20-14', '20-15', '20-16', '20-18', '20-19', '20-20', '20-3', '20-4', '20-5', '20-7', '20-8', '3-1', '3-13', '3-18', '3-19', '3-2', '3-20', 

In [3]:
# 取出每个发射机的IQ数据并进行合并
dataset = generate_tx_datasets(compact_dataset,capture_date_list,tx_list,rx_list,equalized=equalized)

# 转化为DataFrame格式
df = pd.DataFrame(dataset)

In [ ]:
# 生成短时时序图（按顺序）

# 生成差分星座图和轨迹图的函数
def plot_diff_constellation(iq_complex, tx_id, group_id, output_folder, line_color='red', line_width=1.5, scatter_color='blue'):
    # 计算差分星座符号
    diff_symbols = iq_complex[1:] * np.conj(iq_complex[:-1])
    
    # 创建发射机文件夹，如果不存在
    tx_folder = os.path.join(output_folder, str(tx_id))
    if not os.path.exists(tx_folder):
        os.makedirs(tx_folder)
    
    # 创建存放散点图和轨迹图的子文件夹
    scatter_folder = os.path.join(tx_folder, 'scatter_plots')
    trajectory_folder = os.path.join(tx_folder, 'trajectory_plots')
    
    if not os.path.exists(scatter_folder):
        os.makedirs(scatter_folder)
    if not os.path.exists(trajectory_folder):
        os.makedirs(trajectory_folder)
    
    # 绘制差分星座散点图
    plt.figure(figsize=(8, 8))
    plt.scatter(np.real(diff_symbols), np.imag(diff_symbols), color=scatter_color, alpha=0.7)
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)  # 横轴
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)  # 纵轴
    plt.xticks([])  # 去掉x轴刻度
    plt.yticks([])  # 去掉y轴刻度
    plt.grid(False)  # 关闭网格

    # 保存差分星座散点图
    scatter_file = os.path.join(scatter_folder, f'scatter_group_{group_id}.png')
    plt.savefig(scatter_file)
    plt.close()  # 保存后关闭，释放内存

    # 绘制差分轨迹图
    plt.figure(figsize=(8, 8))
    plt.plot(np.real(diff_symbols), np.imag(diff_symbols), color=line_color, linewidth=line_width, alpha=0.7)
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)  # 横轴
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)  # 纵轴
    plt.xticks([])  # 去掉x轴刻度
    plt.yticks([])  # 去掉y轴刻度
    plt.grid(False)  # 关闭网格

    # 保存差分轨迹图
    trajectory_file = os.path.join(trajectory_folder, f'trajectory_group_{group_id}.png')
    plt.savefig(trajectory_file)
    plt.close()  # 保存后关闭，释放内存

# 将图像调整为 ResNet-18 输入格式
def preprocess_image_for_resnet(image_path, target_size=(224, 224)):
    # 读取图像
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 以灰度图模式读取
    image_resized = cv2.resize(image, target_size)  # 调整图像大小
    
    # 如果是灰度图，将其转换为三通道
    image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)
    
    # 标准化：归一化到 [0, 1] 范围，并减去均值
    image_normalized = image_rgb.astype(np.float32) / 255.0  # 将图像像素值归一化到 [0, 1]
    
    # ResNet-18 预处理的均值和标准差
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    # 减去均值并除以标准差进行标准化
    image_normalized = (image_normalized - mean) / std
    
    return image_normalized

# 生成差分星座图和轨迹图的函数（批次处理版），并进行 ResNet 输入标准化
def generate_diff_constellation_and_trajectory_batch(dataset, output_folder, batch_size=10, line_color='red', line_width=1.5, scatter_color='blue'):
    # 将数据按批次分割
    num_tx = len(dataset['node_list'])
    for start_idx in range(0, num_tx, batch_size):
        end_idx = min(start_idx + batch_size, num_tx)
        batch_tx_ids = dataset['node_list'][start_idx:end_idx]
        batch_data = dataset['data'][start_idx:end_idx]
        
        for tx_id, data in zip(batch_tx_ids, batch_data):
            # 处理每个发射机的嵌套数组，reshape为 (N, 256, 2)，每组包含256个IQ信号
            iq_data = data  # (N, 256, 2)
            
            # 对每一组IQ信号进行差分处理并绘图
            for group_id in range(iq_data.shape[0]):  # 遍历每组数据
                # 将IQ信号转换为复数形式
                iq_complex = iq_data[group_id, :, 0] + 1j * iq_data[group_id, :, 1]
                
                # 对每组信号画差分星座图和轨迹图
                plot_diff_constellation(iq_complex, tx_id, group_id, output_folder, line_color, line_width, scatter_color)
                
                # 生成图片路径
                scatter_image_path = os.path.join(output_folder, str(tx_id), 'scatter_plots', f'scatter_group_{group_id}.png')
                trajectory_image_path = os.path.join(output_folder, str(tx_id), 'trajectory_plots', f'trajectory_group_{group_id}.png')
                
                # 预处理图片（调整大小、转换为三通道、标准化）
                scatter_image_processed = preprocess_image_for_resnet(scatter_image_path)
                trajectory_image_processed = preprocess_image_for_resnet(trajectory_image_path)
                
                # 保存预处理后的图片，或者直接返回给训练模型使用
                # 此处可以根据需要保存预处理后的图片，或者返回进行下一步处理

        # 清理内存，处理完一个批次后释放内存
        print(f"Batch {start_idx // batch_size + 1} finished. Memory cleared.")

# 使用示例：生成不同发射机的差分星座图和轨迹图
output_folder = "../IQ_signal_plots"  # 图像输出的根文件夹
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

line_color = 'blue'  # 设置轨迹图的线条颜色
line_width = 2  # 设置轨迹图的线条宽度
scatter_color = 'blue'  # 设置散点图的颜色

# 调用批次处理函数生成图像并进行 ResNet 输入标准化
generate_diff_constellation_and_trajectory_batch(dataset, output_folder, batch_size=10, line_color=line_color, line_width=line_width, scatter_color=scatter_color)


In [ ]:
# 创建长时序轨迹图

# 生成差分星座图和轨迹图的函数
def plot_diff_constellation(iq_complex, tx_id, group_id, output_folder, line_color='red', line_width=1.5, scatter_color='blue'):
    # 计算差分星座符号
    diff_symbols = iq_complex[1:] * np.conj(iq_complex[:-1])
    
    # 创建发射机文件夹，如果不存在
    tx_folder = os.path.join(output_folder, str(tx_id))
    if not os.path.exists(tx_folder):
        os.makedirs(tx_folder)
    
    # 创建存放散点图和轨迹图的子文件夹
    scatter_folder = os.path.join(tx_folder, 'scatter_plots')
    trajectory_folder = os.path.join(tx_folder, 'trajectory_pos')
    
    if not os.path.exists(scatter_folder):
        os.makedirs(scatter_folder)
    if not os.path.exists(trajectory_folder):
        os.makedirs(trajectory_folder)
    
    # 绘制差分星座散点图
    plt.figure(figsize=(8, 8))
    plt.scatter(np.real(diff_symbols), np.imag(diff_symbols), color=scatter_color, alpha=0.7)
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)  # 横轴
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)  # 纵轴
    plt.xticks([])  # 去掉x轴刻度
    plt.yticks([])  # 去掉y轴刻度
    plt.grid(False)  # 关闭网格

    # 保存差分星座散点图
    scatter_file = os.path.join(scatter_folder, f'scatter_group_{group_id}.png')
    plt.savefig(scatter_file)
    plt.close()  # 保存后关闭，释放内存

    # 绘制差分轨迹图
    plt.figure(figsize=(8, 8))
    plt.plot(np.real(diff_symbols), np.imag(diff_symbols), color=line_color, linewidth=line_width, alpha=0.7)
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)  # 横轴
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)  # 纵轴
    plt.xticks([])  # 去掉x轴刻度
    plt.yticks([])  # 去掉y轴刻度
    plt.grid(False)  # 关闭网格

    # 保存差分轨迹图
    trajectory_file = os.path.join(trajectory_folder, f'trajectory_group_{group_id}.png')
    plt.savefig(trajectory_file)
    plt.close()  # 保存后关闭，释放内存

# 将图像调整为 ResNet-18 输入格式
def preprocess_image_for_resnet(image_path, target_size=(224, 224)):
    # 读取图像
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 以灰度图模式读取
    image_resized = cv2.resize(image, target_size)  # 调整图像大小
    
    # 如果是灰度图，将其转换为三通道
    image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)
    
    # 标准化：归一化到 [0, 1] 范围，并减去均值
    image_normalized = image_rgb.astype(np.float32) / 255.0  # 将图像像素值归一化到 [0, 1]
    
    # ResNet-18 预处理的均值和标准差
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    # 减去均值并除以标准差进行标准化
    image_normalized = (image_normalized - mean) / std
    
    return image_normalized

# 生成差分星座图和轨迹图的函数（批次处理版），调整轨迹图连接逻辑
def generate_diff_constellation_and_trajectory_batch(dataset, output_folder, batch_size=10, line_color='red', line_width=1.5):
    # 初始化统计字典
    tx_image_count = defaultdict(int)
    
    # 进度条设置
    total_tx = len(dataset['node_list'])
    main_pbar = tqdm(total=total_tx, desc="总进度", position=0, dynamic_ncols=True)

    for start_idx in range(0, total_tx, batch_size):
        end_idx = min(start_idx + batch_size, total_tx)
        batch_tx_ids = dataset['node_list'][start_idx:end_idx]
        batch_data = dataset['data'][start_idx:end_idx]

        batch_pbar = tqdm(zip(batch_tx_ids, batch_data), 
                        total=len(batch_tx_ids),
                        desc="当前批次",
                        position=1,
                        leave=False)

        for tx_id, data in batch_pbar:
            # 初始化当前TX计数器
            current_count = 0
            
            # 更新进度描述
            batch_pbar.set_postfix_str(f"处理中: {tx_id.ljust(10)} | 已生成: {current_count} 图")

            # 创建文件夹
            tx_folder = os.path.join(output_folder, str(tx_id))
            trajectory_folder = os.path.join(tx_folder, 'trajectory_pos')
            os.makedirs(trajectory_folder, exist_ok=True)

            iq_data = data  # (N, 256, 2)
            num_groups = iq_data.shape[0]

            for batch_idx, batch in group_pbar:
                position_trajectories = defaultdict(list)

                # 收集数据
                for group in batch:
                    iq_complex = group[:, 0] + 1j * group[:, 1]
                    diff_symbols = iq_complex[1:] * np.conj(iq_complex[:-1])
                    
                    for pos in range(len(diff_symbols)):
                        position_trajectories[pos].append(diff_symbols[pos])

                for pos, symbols in pos_pbar:
                    # 生成并保存图像
                    plt.figure(figsize=(8, 8))
                    plt.plot(np.real(symbols), np.imag(symbols), 
                            color=line_color, linewidth=line_width, alpha=0.7)
                    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
                    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
                    plt.xticks([])
                    plt.yticks([])
                    plt.grid(False)
                    
                    trajectory_file = os.path.join(
                        trajectory_folder, 
                        f'trajectory_pos_{pos:03d}_batch_{batch_idx:03d}.png'
                    )
                    plt.savefig(trajectory_file)
                    plt.close()
                    
                    # 更新计数器
                    current_count += 1
                    tx_image_count[tx_id] = current_count
                    batch_pbar.set_postfix_str(f"处理中: {tx_id.ljust(10)} | 已生成: {current_count} 图")

                pos_pbar.close()
            group_pbar.close()

            # 打印当前TX最终统计
            print(f"\n✅ {tx_id} 完成 | 共生成 {current_count} 张轨迹图")
            main_pbar.update(1)
            main_pbar.set_postfix_str(f"最新完成: {tx_id}")

        batch_pbar.close()
    
    # 最终统计报告
    print("\n" + "="*50)
    print("发射机图像生成统计报告：")
    for tx_id, count in tx_image_count.items():
        print(f"• {tx_id.ljust(15)}: {str(count).rjust(5)} 张")
    print("="*50)
    
    main_pbar.close()
        
# 使用示例：生成不同发射机的差分星座图和轨迹图
output_folder = "../../IQ_signal_plots"  # 图像输出的根文件夹
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

line_color = 'blue'  # 设置轨迹图的线条颜色
line_width = 2  # 设置轨迹图的线条宽度
scatter_color = 'blue'  # 设置散点图的颜色

# 调用批次处理函数生成图像并进行 ResNet 输入标准化
generate_diff_constellation_and_trajectory_batch(dataset, output_folder, batch_size=10, line_color=line_color, line_width=line_width)


总进度:   0%|          | 0/150 [00:00<?, ?it/s]

当前批次:   0%|          | 0/10 [00:00<?, ?it/s]

分组处理:   0%|          | 0/12 [00:00<?, ?it/s]

轨迹生成:   0%|          | 0/255 [00:00<?, ?it/s]

轨迹生成:   0%|          | 0/255 [00:00<?, ?it/s]

...

分组处理:   0%|          | 0/15 [00:00<?, ?it/s]

轨迹生成:   0%|          | 0/255 [00:00<?, ?it/s]

轨迹生成:   0%|          | 0/255 [00:00<?, ?it/s]

轨迹生成:   0%|          | 0/255 [00:00<?, ?it/s]

轨迹生成:   0%|          | 0/255 [00:00<?, ?it/s]

In [5]:
# 删除指定文件夹中的图片

import os
import shutil

def clean_trajectory_pos(root_dir="IQ_signal_plots"):
    """
    清理所有TX_ID下的trajectory_pos文件夹中的图片文件
    :param root_dir: 根目录路径（默认为IQ_signal_plots）
    """
    # 支持的图片文件扩展名
    image_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'}
    
    # 遍历根目录下的所有TX_ID文件夹
    for tx_id in os.listdir(root_dir):
        tx_path = os.path.join(root_dir, tx_id)
        
        # 检查是否是有效目录
        if not os.path.isdir(tx_path):
            continue
            
        # 定位trajectory_pos文件夹
        traj_pos_path = os.path.join(tx_path, 'trajectory_pos')
        
        # 如果目标文件夹不存在则跳过
        if not os.path.exists(traj_pos_path):
            print(f"跳过 {tx_id}: trajectory_pos 文件夹不存在")
            continue
            
        # 删除所有图片文件
        deleted_count = 0
        for filename in os.listdir(traj_pos_path):
            file_path = os.path.join(traj_pos_path, filename)
            
            # 检查文件扩展名和类型
            if os.path.splitext(filename)[-1].lower() in image_extensions:
                try:
                    os.remove(file_path)
                    deleted_count += 1
                except Exception as e:
                    print(f"删除失败 {file_path}: {str(e)}")
                    
        print(f"已清理 {tx_id}: 删除 {deleted_count} 个文件")

if __name__ == "__main__":
    # 使用示例（可修改路径）
    clean_trajectory_pos("../../IQ_signal_plots")
    
    # 可选：同时删除空文件夹
    # shutil.rmtree("IQ_signal_plots", ignore_errors=True)  # 慎用！

删除失败 ../../IQ_signal_plots\1-1\trajectory_pos\trajectory_pos_176_batch_001.png: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: '../../IQ_signal_plots\\1-1\\trajectory_pos\\trajectory_pos_176_batch_001.png'
已清理 1-1: 删除 658 个文件
跳过 1-10: trajectory_pos 文件夹不存在
跳过 1-11: trajectory_pos 文件夹不存在
跳过 1-12: trajectory_pos 文件夹不存在
跳过 1-14: trajectory_pos 文件夹不存在
跳过 1-15: trajectory_pos 文件夹不存在
跳过 1-16: trajectory_pos 文件夹不存在
跳过 1-18: trajectory_pos 文件夹不存在
跳过 1-19: trajectory_pos 文件夹不存在
跳过 1-2: trajectory_pos 文件夹不存在
跳过 1-8: trajectory_pos 文件夹不存在
跳过 10-1: trajectory_pos 文件夹不存在
跳过 10-10: trajectory_pos 文件夹不存在
跳过 10-11: trajectory_pos 文件夹不存在
跳过 10-17: trajectory_pos 文件夹不存在
跳过 10-4: trajectory_pos 文件夹不存在
跳过 10-7: trajectory_pos 文件夹不存在
跳过 11-1: trajectory_pos 文件夹不存在
跳过 11-10: trajectory_pos 文件夹不存在
跳过 11-17: trajectory_pos 文件夹不存在
跳过 11-19: trajectory_pos 文件夹不存在
跳过 11-20: trajectory_pos 文件夹不存在
跳过 11-4: trajectory_pos 文件夹不存在
跳过 11-7: trajectory_pos 文件夹不存在
跳过 12-1: trajectory_pos 文件夹不存在
跳过 12-19: trajectory_pos 文件夹不存在
跳过 1